In [5]:
import requests
import xml.etree.ElementTree as ET  # for parsing XML

In [6]:
from dotenv import load_dotenv
import os
load_dotenv()
from time import sleep
from tqdm import tqdm
mongodb_uri = os.environ.get("MONGODB_URI")

In [7]:
base_semester_url = "https://courses.illinois.edu/cisapp/explorer/schedule/2024/spring.xml"

subjects_list = []

subjects_root = ET.fromstring(requests.get(base_semester_url).content)
subjects_xml = subjects_root.find("subjects").findall("subject")

In [8]:
courses_list = []

for subject in tqdm(subjects_xml) : 
    course_root = ET.fromstring(requests.get(subject.attrib["href"]).content)
    course_xml = course_root.find("courses").findall("course")
    for course in course_xml:
        courses_list.append({
            "name" : course.text,
            "number" : course.attrib["id"],
            "code" : subject.attrib["id"],
            "api_link" : course.attrib["href"],
            "year" : 2024,
            "semester" : "spring"
        })

100%|██████████| 191/191 [03:48<00:00,  1.19s/it]


In [9]:
print(courses_list)

[{'name': 'Intro Asian American Studies', 'number': '100', 'code': 'AAS', 'api_link': 'https://courses.illinois.edu/cisapp/explorer/schedule/2024/spring/AAS/100.xml', 'year': 2024, 'semester': 'spring'}, {'name': 'U.S. Race and Empire', 'number': '200', 'code': 'AAS', 'api_link': 'https://courses.illinois.edu/cisapp/explorer/schedule/2024/spring/AAS/200.xml', 'year': 2024, 'semester': 'spring'}, {'name': 'US Racial & Ethnic Politics', 'number': '201', 'code': 'AAS', 'api_link': 'https://courses.illinois.edu/cisapp/explorer/schedule/2024/spring/AAS/201.xml', 'year': 2024, 'semester': 'spring'}, {'name': 'The Politics of Fashion', 'number': '275', 'code': 'AAS', 'api_link': 'https://courses.illinois.edu/cisapp/explorer/schedule/2024/spring/AAS/275.xml', 'year': 2024, 'semester': 'spring'}, {'name': 'Asian American History', 'number': '283', 'code': 'AAS', 'api_link': 'https://courses.illinois.edu/cisapp/explorer/schedule/2024/spring/AAS/283.xml', 'year': 2024, 'semester': 'spring'}, {'na

In [14]:
import pymongo
client = pymongo.MongoClient(mongodb_uri)
db = client[os.environ.get("MONGODB_DB")]

In [16]:
classes = db["classes"]
classes.insert_many(courses_list)